# Project Details

### AIM : 
<b> To implement a language model which will perform better in playing the Hangman game as compared to the traditional models which depend on unigram,bigram and n-gram models</b>

### Creating test beds
##### Corpus Description
<ul>
<li>Name: <b> corncob_lowercase.txt.</b></li>
<li>Description: <b> Contains 58110 lowercase english words</b></li>
</ul>

##### <b>Stratergy :</b> The model will evaluated using the concept of 10-cross validation
<small>In k-fold cross-validation, the original sample is randomly partitioned into k equal size subsamples. Of the k subsamples, a single subsample is retained as the validation data for testing the model, and the remaining k-1 subsamples are used as training data. The cross-validation process is then repeated k times (the folds), with each of the k subsamples used exactly once as the validation data. The k results from the folds can then be averaged (or otherwise combined) to produce a single estimation. The advantage of this method is that all observations are used for both training and validation, and each observation is used for validation exactly once.</small>

In [20]:
import random

words = []
with open("corncob_lowercase.txt", mode="r") as myFile:
    for line in myFile:
        words.append(line.strip())
        
# Randomly shuffling the words to create better and meaningful k-flood samples
random.shuffle(words)


### Personal Details

<ol>
    <li>Name: <b>G V Sandeep</b></li>
    <li>College: <b>BITS - Pilani, Hyderabad Campus</b></li>
    <li>Github: <a href="https://github.com/greetsandeep/">greetsandeep</a></li>
</ol>

This code is open sourced and can be found at : <a href"https://github.com/greetsandeep/ACM_SummerSchool/tree/master/Improvising%20Hangman">Improvising Hangman</a>